In [1]:
lista = 2
if type(lista) != list:
    print('não é uma lista')

não é uma lista


In [2]:
dicio = {'teste1':1,'teste2':2}
for name, value in dicio.items():
    print(name,value)

teste1 1
teste2 2


In [3]:
import sympy as sp

x,p,A = sp.symbols('x,p,A')

f = sp.Function('f')(x)
#A = sp.Function('A')(x)
N = sp.Function('N')(x,p)

expr = sp.Eq(f.diff(x),f)
expr

Eq(Derivative(f(x), x), f(x))

In [4]:
expr.subs(f,A + x*N)

Eq(Derivative(A + x*N(x, p), x), A + x*N(x, p))

In [5]:
print(sp.simplify(expr.subs(f,A + x*N)))

Eq(A + x*N(x, p), x*Derivative(N(x, p), x) + N(x, p))


In [12]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Definindo a classe para o modelo usando Subclassing do Keras
class NeuralODEModel(keras.Model):
    def __init__(self, num_hidden_neurons):
        super(NeuralODEModel, self).__init__()
        # Camada oculta
        self.hidden = keras.layers.Dense(num_hidden_neurons, activation='sigmoid', use_bias=True)
        # Camada de saída
        self.output_layer = keras.layers.Dense(1, activation=None, use_bias=True)

    def call(self, x):
        x = tf.expand_dims(x, -1)  # Adicionar uma dimensão extra
        h = self.hidden(x)
        out = self.output_layer(h)
        return tf.squeeze(out, -1)  # Remover a dimensão extra

# Função de custo
def cost_function(model, x, g0=10, gamma=2):
    with tf.GradientTape() as tape:
        tape.watch(x)
        g_trial = g0 + x * model(x)  # Solução aproximada
        dg_dx = tape.gradient(g_trial, x)  # Derivada da solução
        g_rhs = -gamma * g_trial  # Lado direito da EDO
        cost = tf.reduce_mean(tf.square(dg_dx - g_rhs))  # Erro quadrático médio
    return cost

# Treinamento do modelo
def train_model(model, x, num_iter, learning_rate, g0=10, gamma=2):
    optimizer = tf.keras.optimizers.Adam(learning_rate)

    for i in range(num_iter):
        with tf.GradientTape() as tape:
            cost = cost_function(model, x, g0, gamma)
        gradients = tape.gradient(cost, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        if i % (num_iter // 10) == 0 or i == num_iter - 1:
            print(f"Iteration {i+1}/{num_iter}, Cost: {cost.numpy():.6f}")

# Solução analítica para comparação
def g_analytic(x, gamma=2, g0=10):
    return g0 * np.exp(-gamma * x)

if __name__ == "__main__":
    # Dados de entrada
    N = 10
    x = tf.linspace(0, 1, N)[:,tf.newaxis]

    # Configuração do modelo
    num_hidden_neurons = 10
    num_iter = 1000
    learning_rate = 0.01

    # Criação e treinamento do modelo
    model = NeuralODEModel(num_hidden_neurons)
    train_model(model, x, num_iter, learning_rate)

    # Solução aproximada pelo modelo
    g_nn = g_analytic(x[0]) + x.T * model(x).numpy()

    # Solução analítica
    g_exact = g_analytic(x)

    # Comparação
    max_diff = np.max(np.abs(g_nn - g_exact))
    print(f"Max absolute difference: {max_diff}")

    # Plotagem dos resultados
    import matplotlib.pyplot as plt
    plt.plot(x, g_exact, label='Analytical Solution')
    plt.plot(x, g_nn, label='Neural Network Solution')
    plt.legend()
    plt.xlabel('x')
    plt.ylabel('g(x)')
    plt.title('ODE Solution: Analytical vs Neural Network')
    plt.show()


InvalidArgumentError: cannot compute Mul as input #1(zero-based) was expected to be a double tensor but is a float tensor [Op:Mul] name: 

In [2]:
import tensorflow as tf
print(tf.__version__)  # Deve retornar uma versão compatível, como '2.x'


2.17.0
